# Amazon OpenSearch Serverless와 LangChain으로 빠르게 대화형 검색 구현하기

## Overview

이 워크샵에서는 Amazon OpenSearch Serverless와 LangChain을 활용하여 대화형 검색을 빠르게 구현하는 방법에 대해 알아봅니다. Amazon OpenSearch Serverless는 서버리스 환경에서 OpenSearch를 실행하고 관리하는 데 필요한 모든 기능을 제공합니다. LangChain은 머신러닝 기반의 언어 처리 플랫폼으로, 텍스트 데이터를 효과적으로 분석하고 이해하는 데 도움을 줍니다. 이 두 도구를 결합하면 대화형 검색을 효율적으로 구현할 수 있습니다.

### Amazon OpenSearch Serverless는?

Amazon OpenSearch Serverless는 고성능 검색 및 분석 기능을 제공하며, 사용자가 인프라 관리를 걱정하지 않고 데이터에 집중할 수 있도록 해줍니다. 기본 리소스를 자동으로 프로비저닝하고 확장하여 가장 복잡하고 예측할 수 없는 워크로드에도 빠른 데이터 수집 및 쿼리 응답을 제공합니다. 따라서 클러스터를 구성하고 최적화하는 데 필요한 작업은 없습니다.

Amazon OpenSearch Serverless를 사용하면 쿼리의 빈도나 복잡성, 분석 대상 데이터의 양 등 예측하기 어려운 요소를 고려할 필요가 없습니다. 인프라 관리 대신 OpenSearch를 활용하여 데이터 탐색과 인사이트 추출에 집중할 수 있습니다. 또한, 익숙한 API를 통해 데이터를 로드하고 쿼리하며, OpenSearch Dashboards를 이용해 대화형 데이터 분석 및 시각화를 할 수 있습니다.

### **LangChain**

LangChain은 대규모 언어 모델(LLM)을 활용하여 데이터에서 인사이트를 추출하고, 질문에 답변하며, 새로운 콘텐츠를 생성하는 등의 작업을 수행할 수 있는 프레임워크입니다. LangChain은 LLM을 다양한 소스의 데이터와 연결하고, 체인을 구성하여 복잡한 작업을 수행할 수 있도록 지원합니다.

LangChain은 다음과 같은 주요 기능을 제공합니다:

1. **데이터 로딩**: LangChain은 다양한 유형의 데이터(PDF, CSV, 웹페이지 등)를 로드하고 LLM에 적합한 형식으로 변환할 수 있습니다.
2. **체인 구성**: LangChain을 사용하면 여러 개의 LLM과 다른 유형의 체인(순차적, 반복적, 메모리 등)을 구성할 수 있습니다.
3. **검색 및 질의 응답**: LangChain은 벡터 데이터베이스와 통합되어 관련 문서를 검색하고, 검색 결과를 기반으로 질문에 답변할 수 있습니다.
4. **에이전트**: LangChain은 에이전트라는 개념을 제공하며, 에이전트는 여러 도구와 상호작용하여 복잡한 작업을 수행할 수 있습니다.
5. **메모리**: LangChain은 대화 기록, 중간 결과 등을 저장하고 활용할 수 있는 메모리 기능을 제공합니다.

### **Amazon OpenSearch Serverless와 LangChain 통합**

Amazon OpenSearch Serverless와 LangChain을 통합하면 다음과 같은 이점이 있습니다:

1. **대규모 데이터 처리**: Amazon OpenSearch Serverless는 대규모 데이터를 효율적으로 저장하고 검색할 수 있습니다. LangChain은 이 데이터를 로드하고 LLM과 연결하여 인사이트를 추출할 수 있습니다.
2. **벡터 검색**: Amazon OpenSearch Serverless 벡터 엔진을 사용하면 LangChain에서 벡터 데이터베이스를 활용하여 관련 문서를 효율적으로 검색할 수 있습니다.
3. **대화형 검색 경험**: LangChain의 질의 응답 기능과 Amazon OpenSearch Serverless의 검색 기능을 결합하면 자연어 질문에 대한 대화형 검색 경험을 제공할 수 있습니다.
4. **확장성**: Amazon OpenSearch Serverless는 서버리스 아키텍처를 기반으로 하므로 워크로드에 따라 자동으로 확장되며, LangChain은 이 확장된 리소스를 활용할 수 있습니다.
5. **비용 효율성**: Amazon OpenSearch Serverless는 사용한 만큼만 비용을 지불하는 서버리스 모델을 따르므로 비용 효율적입니다.

이 워크샵에서는 Amazon OpenSearch Serverless와 LangChain을 통합하여 대화형 검색 애플리케이션을 구축하는 과정을 단계별로 안내합니다. 데이터 로딩, 벡터 인덱싱, LLM 통합, 질의 응답 등의 주요 기능을 다룰 예정입니다.

## 사전 준비

필요한 패키지 설치 및 임포트합니다

In [ ]:
%pip install -U --quiet boto3
%pip install -U --quiet  opensearch-py langchain-community
%pip install -U --quiet requests-aws4auth
%pip install -U --quiet langchain_aws

In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import OpenSearchVectorSearch
from langchain_text_splitters import CharacterTextSplitter
import textwrap
import json

boto3 세션 정보를 가져와 Amazon OpenSearch Serverless 클라이언트 생성합니다.

In [ ]:
import boto3

session = boto3.Session()
aoss_client = session.client(service_name="opensearchserverless")

컬렉션 이름과 리전 변수를 초기화합니다. 이전 단계에서 생성한 Amazon Bedrock Knowledge Base의 Vector Store를 재사용해보겠습니다. 

In [ ]:
response = aoss_client.list_collections()

collection_name = response["collectionSummaries"][0]["name"]
collection_id = response["collectionSummaries"][0]["id"]
collection_name, collection_id

In [ ]:
session = boto3.Session()
region_name = session.region_name
region_name

컬렉션을 생성하기 위해서는 반드시 컬렉션 이름과 일치하는 리소스 패턴을 가진 암호화 정책(encryption policy)와 네트워크 정책(Network Policy)을 가지고 있어야 합니다. 아래 코드는 위의 `collection_name`에 맞는 암호화 / 네트워크 정책이 있는지 확인하고 없으면 생성합니다.

### SageMaker Notebook의 IAM Role을 OpenSearch Collection Data Access에 추가하기

이 과정에서 사용하고 있는 Amazon OpenSearch Serverless Collection은 이전 과정의 Knowledge Base 생성에서 자동으로 생성되었기 때문에 현재 SageMaker Notebook은 데이터에 접근할 권한이 없습니다. 따라서 현재 사용중인 SageMaker Notebook의 IAM Role을 Collection의 데이터를 사용할 수 있도록 data access policy를 생성해야 합니다. 

먼저 현재 SageMaker Notebook의 IAM Role을 가져옵니다.

In [ ]:
import boto3
import botocore

# Create a session with boto3
session = boto3.Session()

# Get the STS client
sts_client = session.client("sts")

# Get the caller identity
caller_identity = sts_client.get_caller_identity()

# Get the assumed role ARN
assumed_role_arn = caller_identity["Arn"]

# Get the role name from the assumed role ARN
role_name = assumed_role_arn.split("/")[1]

# Get the IAM client
iam_client = session.client("iam")

is_user = False

# Get the role details
try:
    role = iam_client.get_role(RoleName=role_name)
    iam_role_arn = role["Role"]["Arn"]

    print(f"The IAM role assumed now is: {assumed_role_arn}")
    print(f"The IAM role attached to this SageMaker notebook instance is: {iam_role_arn}")
except botocore.exceptions.ClientError as e:
    print(f"Error: {e}")

해당 Role로 Collection Data Access Policy를 생성합니다.

# Define the policy
policy_document = f"""
[{{
    "Rules": [
        {{
            "ResourceType": "collection",
            "Resource": ["collection/{collection_name}"],
            "Permission": [
                "aoss:*"
            ]
        }}, 
        {{
            "ResourceType": "index",
            "Resource": ["index/{collection_name}/*"],
            "Permission": [
                "aoss:*"
            ]
        }}
    ],
    "Principal": ["{iam_role_arn}"]
}}]
"""

# Create the access policy
response = aoss_client.create_access_policy(
    name="sagemaker-notebook-access-policy",
    type="data",
    description="Data access policy for SageMaker notebook",
    policy=policy_document
)

print("Access policy created:", response['accessPolicyDetail']['name'])

## AOSS 클라이언트 생성 및 인증 정보 초기화

이번 과정은 SageMaker Notebook 내의 `00.langchin_aoss.ipynb` 상에서 진행됩니다. 먼저 Amazon OpenSearch Serverless에 접근하기 위한 인증 정보를 초기화합니다.

In [ ]:
credentials = session.get_credentials()
credentials

In [ ]:
from requests_aws4auth import AWS4Auth

service = "aoss"  # must set the service as 'aoss'
awsauth = AWS4Auth(
    credentials.access_key,
    credentials.secret_key,
    region_name,
    service,
    session_token=credentials.token,
)

LangChain의 CSVLoader를 사용하여 `movies.csv` 파일을 읽어, 각 문서를 일정 크기의 조각(chunk)으로 분할하는 `CharacterTextSplitter`를 사용합니다. 이렇게 분할된 문서들이 `docs` 변수에 저장되며, 마지막으로 `len(docs)`를 통해 분할된 문서의 개수를 확인합니다. 현재 movies.csv 파일에는 모두 1000개의 영화에 대한 정보가 들어있으므로 docs의 길이는 1000이 됩니다. 

`CSVLoader`는 CSV 파일로부터 데이터를 로드하는 역할을 합니다. 이 클래스는 파일 경로를 인자로 받아, 해당 파일을 읽고 각 행을 하나의 문서로 변환합니다. 각 문서는 딕셔너리 형태로 저장되며, CSV 파일의 각 열은 딕셔너리의 키가 되고, 해당 행의 값은 딕셔너리의 값이 됩니다. 이렇게 변환된 문서들은 리스트로 묶여 반환됩니다. 따라서 `CSVLoader`를 사용하면, CSV 형식의 데이터를 쉽게 LangChain에서 사용할 수 있는 형식으로 변환할 수 있습니다.

CSVLoader를 사용했기 때문에 chunk_size가 1000이라 하더라도 chunk_size 기준이 아닌 CSV 형식에 맞춰서 구분 되게 됩니다.

In [ ]:
from langchain_community.document_loaders import CSVLoader
from langchain_community.embeddings import BedrockEmbeddings
from opensearchpy import RequestsHttpConnection

loader = CSVLoader("./data/movies.csv")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)


len(docs)

Amazon의 Bedrock 서비스를 사용하여 문서 내용을 벡터로 임베딩합니다. BedrockEmbeddings 클래스를 사용하여 amazon.titan-embed-text-v2:0 모델을 로드하고, 잘 동작하는지 테스트하기 위해 embed_query를 호출해봅니다. 

In [ ]:
embeddings = BedrockEmbeddings(
    model_id="amazon.titan-embed-text-v2:0",
    region_name=region_name,
)

# Test embedding models
vector = embeddings.embed_query("This is a content of the document")
len(vector)

OpenSearchVectorSearch 클래스를 사용하여 문서 데이터를 OpenSearch 서비스에 인덱싱합니다. from_documents 메서드를 호출하여 docs 리스트에 있는 문서들을 embeddings 객체를 사용하여 벡터로 임베딩한 뒤, aoss_host에 지정된 OpenSearch 서비스 URL과 awsauth 인증 정보를 사용하여 "top_movies" 인덱스에 bulk 방식으로 인덱싱합니다.

In [ ]:
%%time

aoss_host = f"https://{collection_id}.{region_name}.aoss.amazonaws.com"

docsearch = OpenSearchVectorSearch.from_documents(
    docs,
    embeddings,
    opensearch_url=aoss_host,
    http_auth=awsauth,
    timeout=60,
    connection_class=RequestsHttpConnection,
    index_name="top_movies",
    engine="faiss",
    bulk_size=20000,
)

데이터가 잘 인덱싱되었는지 `similarity_search`를 호출하여 확인합니다. 

In [ ]:
docs = docsearch.similarity_search(
    "건축학개론 줄거리를 알려줘",
    k=10,
    search_type="script_scoring",
)

docs

as_retriever 메서드를 호출하여 docsearch 객체를 검색기(retriever)로 변환합니다. 이렇게 생성된 검색기는 벡터 검색을 수행할 수 있습니다. search_kwargs 매개변수를 통해 검색 옵션을 지정할 수 있습니다. 여기서는 "k": 10을 설정하여 상위 10개의 결과를 반환하도록 지정했습니다. 따라서 retriever 객체는 docsearch에 인덱싱된 문서들 중에서 주어진 쿼리와 가장 유사한 상위 10개의 문서를 반환할 수 있는 검색기 역할을 합니다.

In [ ]:
retriever = docsearch.as_retriever(search_kwargs={"k": 10})

ChatBedrock 클래스를 사용하여 Amazon Bedrock에서 제공하는 Claude 3 Sonnet 모델을 로드합니다. 그리고 ConversationBufferWindowMemory 클래스를 사용하여 대화 기록을 저장할 메모리 객체를 생성합니다.이렇게 생성된 llm과 memory 객체를 사용하여 대화 검색 체인을 구성할 수 있습니다.

In [ ]:
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferWindowMemory
from langchain_aws import ChatBedrock

model_kwargs = {  # anthropic
    "anthropic_version": "bedrock-2023-05-31",
    "max_tokens": 2048,
    "temperature": 0,
}

llm = ChatBedrock(
    model_id="anthropic.claude-3-sonnet-20240229-v1:0",  # 파운데이션 모델 지정
    model_kwargs=model_kwargs,
    region_name=region_name,
    streaming=True,
)  # Claude 속성 구성

memory = ConversationBufferWindowMemory(memory_key="chat_history", k=10, return_messages=True)

RAG을 구성하지 않고 환각을 일으켜보겠습니다.

In [ ]:
from langchain_core.messages import HumanMessage

query_text = "건축학개론 줄거리를 알려줘"
messages = [HumanMessage(content=query_text)]

print(textwrap.fill(llm.invoke(messages).content, 80))

## 프롬프트 정의

LangChain 라이브러리의 PromptTemplate을 사용하여 대화형 AI 시스템에 사용할 프롬프트 템플릿을 정의합니다. 이 템플릿을 사용하면 대화형 AI 시스템이 영화 목록과 사용자 질문을 기반으로 적절한 답변을 생성할 수 있습니다.

In [ ]:
from langchain import PromptTemplate

prompt_template = """


Human: Here is the list of movies, inside <movies></movies> XML tags.

<movies>
{context}
</movies>

Only using the contex as above, answer the following question with the rules as below:
    - Don't insert XML tag such as <context> and </context> when answering.
    - Write as much as you can
    - Be courteous and polite
    - Only answer the question if you can find the answer in the context with certainty.
    - Answered in list format
    - Always put a short and concise explanation on why you are recommending this movies.

You are a best movie reviewer in Korea. Please explain a movies from the list above.

Question:
{question}

If the answer is not in the context, just say "추천해드릴만한 영화가 없습니다."


Assistant:"""

PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

LangChain 라이브러리의 PromptTemplate을 사용하여 대화 기록과 후속 질문을 기반으로 독립적인 단일 질문을 생성하는 프롬프트 템플릿을 정의합니다.이 템플릿을 사용하면 대화형 AI 시스템이 이전 대화 기록과 후속 질문을 기반으로 독립적인 단일 질문을 생성할 수 있습니다.

In [ ]:
condense_template = """
Generate one standalone question based on the instructions.

<instrunctions>
- You will be given the following conversation between <chat-history> and </chat-history>
- You will be given the following follow up question between <follow-up-question> and </follow-up-question>
- Standalone question should have summary of the previous questions and answers.
</instructions>

<chat-history>
{chat_history}
</chat-history>

<follow-up-question>
{question}
</follow-up-question>

standalone question:
"""

CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(condense_template)

## 대화형 검색 Chain 구성하기

이전에 설정한 llm, retriever, memory 객체와 PROMPT, CONDENSE_QUESTION_PROMPT를 사용하여 대화형 검색 체인(Conversational Retrieval Chain)을 생성합니다. 

In [ ]:
memory.clear()

conversation_with_retrieval = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=retriever,
    memory=memory,
    combine_docs_chain_kwargs={"prompt": PROMPT},
    condense_question_prompt=CONDENSE_QUESTION_PROMPT,
    # verbose=True,
)

RAG을 구성하지 않고 질문을 해 환각을 일으켜 보겠습니다.

## 대화형 검색 시스템에게 질문해보기

앞서 생성한 conversation_with_retrieval 대화형 검색 체인을 사용하여 첫 번째 질문에 대한 답변을 생성하고 출력합니다.

In [ ]:
first_question = "영화 건축학개론의 줄거리가 뭐야?"
chat_response = conversation_with_retrieval.invoke({"question": first_question})

print(textwrap.fill(chat_response["answer"], 80))

이전 대화의 맥락을 기반으로 새로운 질문에 대한 답변을 생성하고 출력합니다. 사용자가 입력한 "그 영화 평점은?"이라는 질문은 이전 대화에서 언급된 "건축학개론" 영화에 대한 평점을 묻는 질문입니다. 대화형 검색 체인은 이전 대화 맥락을 고려하여 관련 정보를 검색하고 적절한 답변을 생성합니다. 답변은 이전 대화에서 언급된 "건축학개론" 영화의 평점에 대한 내용일 것입니다

In [ ]:
second_question = "그 영화 평점은?"
chat_response = conversation_with_retrieval.invoke({"question": second_question})

print(textwrap.fill(chat_response["answer"], 80))

이전 대화 맥락을 바탕으로 "비슷한 장르의 다른 영화는?"이라는 질문에 대한 답변을 생성하고 출력합니다.  대화형 검색 체인은 이전 대화에서 언급된 영화의 장르를 파악하고, 그와 유사한 장르의 다른 영화를 추천할 수 있습니다. 즉 "건축학 개론과 비슷한 장르의 다른 영화는"에 대한 답변이 출력될 것입니다.

In [ ]:
third_question = "비슷한 장르의 다른 영화는?"
chat_response = conversation_with_retrieval.invoke({third_question})

print(textwrap.fill(chat_response["answer"], 80))